In [ ]:
from collections import deque

#function that sets-up a nested dictionary where the resistor name is the key for the outer dictionary 
#and the information on nodes and resistance are key-value pairs as shown below
def dictResistors(resList, r_info):
    key = r_info[0]
    resList[key] = {
        "V1" : r_info[1],
        "V2" : r_info[2],
        "resistance" : r_info[3]
    }

#function that sets-up the resistor network as a graph using dictionary
#adjacency list using dictionary
#stores parallel connections as appended rname:rvalue pair in the inner dictionary
#reference: ChatGPT
def add_edge(adjacency_list, rname, u, v, rvalue):
    if u in adjacency_list:
        if v in adjacency_list[u]:
            adjacency_list[u][v].append({rname: rvalue}) 
        else:
            adjacency_list[u][v] = [{rname: rvalue}]
    else:
        adjacency_list[u] = {
            v: [{rname: rvalue}]
        }
    
    if v in adjacency_list:
        if u in adjacency_list[v]:
            adjacency_list[v][u].append({rname: rvalue}) 
        else:
            adjacency_list[v][u] = [{rname: rvalue}]
    else:
        adjacency_list[v] = {
            u: [{rname: rvalue}]
    }
        
#function that finds the shortest path using BFS from a start vertex to end vertex
#reference: ChatGPT
def shortest_path(graph, start_vertex, end_vertex):
    if start_vertex == end_vertex:
        return [start_vertex]

    visited = set()
    visited.add(start_vertex)

    queue = deque([(start_vertex, [])])

    while queue:
        current_vertex, path = queue.popleft()

        if current_vertex == end_vertex:
            return path + [current_vertex]

        for neighbor in graph[current_vertex]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [current_vertex]))
    return None

#takes in the first line of user input
N, Q = map(int, input().split())

#transforms the input into a list of resistors called 'resList' by calling def dictResistors
resList = {}
for i in range(N):
    r_info = input().split()
    dictResistors(resList, r_info)

#transforms the data in the resList into a graph called 'resGraph' by calling def add_edge
resGraph = {}
for key in resList:
    add_edge(resGraph, key, resList[key]["V1"], resList[key]["V2"], resList[key]["resistance"])

    
#evaluates the two resistors in the query
for j in range(Q):
    res1, res2 = input().split()
    vertices_res1 = [resList[res1]["V1"], resList[res1]["V2"]]
    vertices_res2 = [resList[res2]["V1"], resList[res2]["V2"]]
    #parallel connection test
    if(set(vertices_res1) == set(vertices_res2)):
        print("PARALLEL")
    else:
        #series connection test
        series = 1
        if vertices_res1 == ['Vdd', 'GND'] or vertices_res2 == ['Vdd', 'GND']:
            series = 0
        else:
            path = shortest_path(resGraph, vertices_res1[1],  vertices_res2[0])
            #print(path)
            for i in path:
                #case1
                if len(path)==1 and (i=="Vdd" or i=="GND"):
                    series = 0
                    break
                #case2
                if i != path[0] and (i == "Vdd" or i == "GND"):
                    series = 0
                    break
                else:
                    #case3
                    if len(resGraph[i].keys()) > 2:
                        series = 0
                        break
                    #case4
                    for neighbors in resGraph[i]:
                        if len(resGraph[i][neighbors]) >= 2:
                            series = 0
                            break
                
        if series == 1: 
            print("SERIES")
        else:
            print("NEITHER")
    